## Criação de Dataframe

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

# Cria uma instância de SparkSession, que é a entrada para usar o Spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()
    
spark

In [3]:
#Carregando dados em um Dataframe e Listando
from datetime import datetime, date
from pyspark.sql import Row

datf=spark.createDataFrame([
    Row(Cidade='Salvador', Populacao=6., Time='Bahia', Dat_cadastro=date(2021, 10, 10)),
    Row(Cidade='Sao Paulo',Populacao=45., Time='Sao Paulo', Dat_cadastro=date(2021, 5, 5)),
    Row(Cidade='Recife', Populacao=3., Time='Sport',Dat_cadastro=date(2021, 7, 2)),
    Row(Cidade='Maceio', Populacao=2., Time='CSA', Dat_cadastro=date(2021, 8, 1)),
])

datf.toPandas()

,Cidade,Populacao,Time,Dat_cadastro
0,Salvador,6.0,Bahia,2021-10-10
1,Sao Paulo,45.0,Sao Paulo,2021-05-05
2,Recife,3.0,Sport,2021-07-02
3,Maceio,2.0,CSA,2021-08-01


In [4]:
#Exibindo os dados do Dataframe

datf.show()

+---------+---------+---------+------------+
|   Cidade|Populacao|     Time|Dat_cadastro|
+---------+---------+---------+------------+
| Salvador|      6.0|    Bahia|  2021-10-10|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|
|   Recife|      3.0|    Sport|  2021-07-02|
|   Maceio|      2.0|      CSA|  2021-08-01|
+---------+---------+---------+------------+



In [5]:
# Exibindo apenas dois registros no Dataframe
datf.show(2)

+---------+---------+---------+------------+
|   Cidade|Populacao|     Time|Dat_cadastro|
+---------+---------+---------+------------+
| Salvador|      6.0|    Bahia|  2021-10-10|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|
+---------+---------+---------+------------+
only showing top 2 rows



In [6]:
# Exibindo tres registros do Dataframe na vertical
datf.show(3, vertical=True)

-RECORD 0------------------
 Cidade       | Salvador   
 Populacao    | 6.0        
 Time         | Bahia      
 Dat_cadastro | 2021-10-10 
-RECORD 1------------------
 Cidade       | Sao Paulo  
 Populacao    | 45.0       
 Time         | Sao Paulo  
 Dat_cadastro | 2021-05-05 
-RECORD 2------------------
 Cidade       | Recife     
 Populacao    | 3.0        
 Time         | Sport      
 Dat_cadastro | 2021-07-02 
only showing top 3 rows



In [7]:
# Exibindo todos os dados com o collect()
datf.collect()

[Row(Cidade='Salvador', Populacao=6.0, Time='Bahia', Dat_cadastro=datetime.date(2021, 10, 10)),
 Row(Cidade='Sao Paulo', Populacao=45.0, Time='Sao Paulo', Dat_cadastro=datetime.date(2021, 5, 5)),
 Row(Cidade='Recife', Populacao=3.0, Time='Sport', Dat_cadastro=datetime.date(2021, 7, 2)),
 Row(Cidade='Maceio', Populacao=2.0, Time='CSA', Dat_cadastro=datetime.date(2021, 8, 1))]

In [8]:
# Sumarizando os dados a partir de determinados campos
datf.select("Cidade", "Populacao").describe().show()

+-------+---------+-----------------+
|summary|   Cidade|        Populacao|
+-------+---------+-----------------+
|  count|        4|                4|
|   mean|     NULL|             14.0|
| stddev|     NULL|20.73644135332772|
|    min|   Maceio|              2.0|
|    max|Sao Paulo|             45.0|
+-------+---------+-----------------+



In [9]:
# Exibindo o Schema do Dataframe
datf.printSchema()

root
 |-- Cidade: string (nullable = true)
 |-- Populacao: double (nullable = true)
 |-- Time: string (nullable = true)
 |-- Dat_cadastro: date (nullable = true)



In [10]:
# Carregando o modulo de funções, expecificamente a função Upper
from pyspark.sql.functions import upper

datf.withColumn('Cidade_U', upper(datf.Cidade)).show()

+---------+---------+---------+------------+---------+
|   Cidade|Populacao|     Time|Dat_cadastro| Cidade_U|
+---------+---------+---------+------------+---------+
| Salvador|      6.0|    Bahia|  2021-10-10| SALVADOR|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|SAO PAULO|
|   Recife|      3.0|    Sport|  2021-07-02|   RECIFE|
|   Maceio|      2.0|      CSA|  2021-08-01|   MACEIO|
+---------+---------+---------+------------+---------+



In [11]:
# Filtrando a Cidade de Salvador nos dados
datf.filter(datf.Cidade == "Salvador").show()

+--------+---------+-----+------------+
|  Cidade|Populacao| Time|Dat_cadastro|
+--------+---------+-----+------------+
|Salvador|      6.0|Bahia|  2021-10-10|
+--------+---------+-----+------------+



In [12]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
datf.createOrReplaceTempView("Dados")
spark.sql("select count(*) from Dados").show()

+--------+
|count(1)|
+--------+
|       4|
+--------+



## Transformando um RDD em Dataframe

In [13]:
# carregando os dados sobre Capital de paises
pais = [("Brasil",10000),("Argentina",20000),("Australia",35000),("Italia",40000),("Egito",65000),("Mexico",80000)]
rddpais= spark.sparkContext.parallelize(pais)

In [14]:
# convertendo os dados do RDD para Dataframe com a operação toDF() 
dataframerdd= rddpais.toDF()

In [15]:
# Exibindo os dados do Dataframe
dataframerdd.show()

+---------+-----+
|       _1|   _2|
+---------+-----+
|   Brasil|10000|
|Argentina|20000|
|Australia|35000|
|   Italia|40000|
|    Egito|65000|
|   Mexico|80000|
+---------+-----+



In [16]:
# Criando o schema das colunas dos campos do Dataframe
Colunas = ["Pais","Total_capital(Bilhoes)"]
dataframerdd2= rddpais.toDF(Colunas)
dataframerdd2.printSchema()
dataframerdd2.show(truncate=False)

root
 |-- Pais: string (nullable = true)
 |-- Total_capital(Bilhoes): long (nullable = true)

+---------+----------------------+
|Pais     |Total_capital(Bilhoes)|
+---------+----------------------+
|Brasil   |10000                 |
|Argentina|20000                 |
|Australia|35000                 |
|Italia   |40000                 |
|Egito    |65000                 |
|Mexico   |80000                 |
+---------+----------------------+

